**Générer le transformer**

In [1]:
import torchvision 
import torch

weights = torchvision.models.ResNet101_Weights.DEFAULT
transformer = weights.transforms()
transformer

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

**Générer les dataloader**

In [2]:
from make_dataloader import make_dataloader
from pathlib import Path

image_path=Path("data/pizza_steak_sushi/")

test_path=image_path/"test"
train_path=image_path/"train"


train_dl, test_dl, class_names = make_dataloader(train_path=train_path,
                test_path=test_path,
                transformer=transformer,
                batch_size=32)


**Générer le modèle**

In [3]:
from torchinfo import summary
from torch import nn


device= "cuda" if torch.cuda.is_available() else "cpu"

model = torchvision.models.resnet101(pretrained=True).to(device)

summary(model=model,
        input_size=(1,3,224,224),
        col_names=['input_size','output_size','num_params','trainable'],
        row_settings=['var_names'])

for param in model.parameters():
    param.requires_grad=False

model.fc=nn.Sequential(
    nn.Linear(in_features=2048, out_features=3, bias=True))

C:\Users\Adam\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Adam\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Adam\AppData\Roaming\Python\Python39\site-packages\torchinfo\torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_

**Générer l'optimizer, la loss_fn et le writer**

In [4]:
from train_modele import train, train_step, test_step
from torch.utils.tensorboard import SummaryWriter
import mlflow
from datetime import datetime
import os

##################################################################
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()


##################################################################

date = datetime.now().strftime("%Y-%m-%d")

log_dir = os.path.join("runs",date,"test")
summary=SummaryWriter(log_dir=log_dir)

##################################################################


**Entraîner le modèle**

In [6]:
training_model= train(model=model,
                      train_dataloader=train_dl,
                      test_dataloader=test_dl,
                      device=device,
                      optimizer=optimizer,
                      loss_fn=loss_fn,
                      epochs=5,
                      writer=summary,
                      )

 20%|██        | 1/5 [00:19<01:18, 19.71s/it]

Epoch: 1 | train_loss: 0.6051 | train_acc: 0.7305 | test_loss: 0.5417 | test_acc: 0.8030


 40%|████      | 2/5 [00:38<00:57, 19.03s/it]

Epoch: 2 | train_loss: 0.6517 | train_acc: 0.6367 | test_loss: 0.4137 | test_acc: 0.9593


 60%|██████    | 3/5 [00:57<00:38, 19.17s/it]

...

Epoch: 4 | train_loss: 0.4948 | train_acc: 0.8164 | test_loss: 0.5791 | test_acc: 0.7415


100%|██████████| 5/5 [01:36<00:00, 19.22s/it]

Epoch: 5 | train_loss: 0.4530 | train_acc: 0.7617 | test_loss: 0.3096 | test_acc: 0.9688



2023/05/20 16:10:44 WARNING mlflow.utils.requirements_utils: Found torch version (2.0.0+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2023/05/20 16:10:48 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.15.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.15.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


MlflowException: The configured tracking uri scheme: 'file' is invalid for use with the proxy mlflow-artifact scheme. The allowed tracking schemes are: {'http', 'https'}